In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 24}) # update font size

x_labels = [r'$0$', r'$\frac{\pi}{8}$', r'$\frac{\pi}{4}$', r'$\frac{3\pi}{8}$',
            r'$\frac{\pi}{2}$', r'$\frac{5\pi}{8}$', r'$\frac{3\pi}{4}$',
            r'$\frac{7\pi}{8}$', r'$\pi$']
x_locations = [np.pi*i/8 for i in range(9)]

L = 2**18

In [ ]:
def to_dB(freq_response):
    return 20*np.log10(abs(freq_response))

def lowpass(omega_c, N, window=np.ones):
    alpha = (N-1)/2
    g_n = np.array([np.sin(omega_c*(n-alpha))/(np.pi*(n-alpha)) if n!=alpha else omega_c/np.pi for n in range(N)])
    h_n = g_n*window(N)
    return h_n

def highpass(omega_c, N, window=np.ones):
    alpha = (N-1)/2
    g_lpf_n = lowpass(omega_c, N, window=window)
    g_hpf_n = -g_lpf_n
    if N % 2 == 1:
        g_hpf_n[int(alpha)] += 1 # including delta[n] term (have to shift since g_lpf_n is already shifted)
    h_n = g_hpf_n*window(N)
    return h_n

def bandpass(omega_l, omega_u, N, window=np.ones):
    alpha = (N-1)/2
    omega_bar = (omega_l+omega_u)/2
    omega_c = (omega_u-omega_l)/2
    g_lpf_n = lowpass(omega_c, N, window=window)
    n = np.arange(N)
    # also shift cosine since we are using shifted window from lpf function
    g_bpf_n = 2*g_lpf_n*np.cos(omega_bar*(n-alpha))
    h_n = g_bpf_n*window(N)
    return h_n

def bandstop(omega_l, omega_u, N, window=np.ones):
    alpha = (N-1)/2
    omega_bar = (omega_l+omega_u)/2
    omega_c = (omega_u-omega_l)/2
    g_lpf_n = lowpass(omega_c, N, window=window)
    n = np.arange(N)
    # also shift cosine since we are using shifted window from lpf function
    g_bpf_n = 2*g_lpf_n*np.cos(omega_bar*(n-alpha))
    g_bsf_n = -g_bpf_n
    # check if filter is odd-length
    if N % 2 == 1:
        g_bsf_n[int(alpha)] += 1 # including delta[n] term
    h_n = g_bsf_n*window(N)
    return h_n

In [ ]:
omega_c = np.pi/2
window = np.ones
lengths = [11, 25, 51, 101]
for N in lengths:
    h_n = lowpass(omega_c, N, window)

    H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
    omega = np.linspace(0, np.pi, len(H_omega))
    plt.figure(figsize=(24, 6))
    plt.subplot(121)
    plt.plot(omega, to_dB(H_omega))
    plt.xlabel(r'$\omega$')
    plt.xticks(x_locations, x_labels)
    plt.ylabel(r'$|H(\omega)|$')
    plt.title(r'$N={}$'.format(N))
    plt.subplot(122)
    plt.plot(omega, np.angle(H_omega))
    plt.xlabel(r'$\omega$')
    plt.xticks(x_locations, x_labels)
    plt.ylabel(r'$\angle H(\omega)$')

In [ ]:
omega_c = np.pi/3
N = 51
window = np.ones
h_n = highpass(omega_c, N, window)

H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 12))
plt.subplot(121)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(122)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')

In [ ]:
omega_l = np.pi/3
omega_u = 2*np.pi/3
window = np.ones
N = 51
h_n = bandpass(omega_l, omega_u, N, window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 12))
plt.subplot(121)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(122)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')

In [ ]:
omega_l = np.pi/3
omega_u = 2*np.pi/3
window = np.ones
N = 101
h_n = bandstop(omega_l, omega_u, N, window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 12))
plt.subplot(121)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(122)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')

# Comparing window types

In [ ]:
# possible window types:
# np.ones, np.bartlett, np.hamming, np.hanning, np.blackman

# bandpass filter parameters
omega_l = np.pi/3
omega_u = 2*np.pi/3
N = 25

# try different windows

# rectangular
window = np.ones
h_n = bandpass(omega_l, omega_u, N, window=window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 6))
plt.subplot(131)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(132)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')
plt.ylim([-np.pi, np.pi])
plt.subplot(133)
plt.stem(window(N))
plt.title('Rectangular window')
plt.xlabel(r'$n$')
plt.ylabel(r'$w[n]$')

# hamming
window = np.hamming
h_n = bandpass(omega_l, omega_u, N, window=window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 6))
plt.subplot(131)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(132)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')
plt.ylim([-np.pi, np.pi])
plt.subplot(133)
plt.stem(window(N))
plt.title('Hamming window')
plt.xlabel(r'$n$')
plt.ylabel(r'$w[n]$')

# blackman
window = np.blackman
h_n = bandpass(omega_l, omega_u, N, window=window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 6))
plt.subplot(131)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(132)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')
plt.ylim([-np.pi, np.pi])
plt.subplot(133)
plt.stem(window(N))
plt.title('Blackman window')
plt.xlabel(r'$n$')
plt.ylabel(r'$w[n]$')

# bartlett
window = np.bartlett
h_n = bandpass(omega_l, omega_u, N, window=window)
H_omega = np.fft.rfft(h_n, L) # only look for omega 0 to pi
omega = np.linspace(0, np.pi, len(H_omega))
plt.figure(figsize=(24, 6))
plt.subplot(131)
plt.plot(omega, to_dB(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$|H(\omega)|$')
plt.subplot(132)
plt.plot(omega, np.angle(H_omega))
plt.xlabel(r'$\omega$')
plt.xticks(x_locations, x_labels)
plt.ylabel(r'$\angle H(\omega)$')
plt.ylim([-np.pi, np.pi])
plt.subplot(133)
plt.stem(window(N))
plt.title('Bartlett window')
plt.xlabel(r'$n$')
plt.ylabel(r'$w[n]$')